In [1]:
import os
%pwd

'd:\\Code\\mlops\\10.endtoend\\endtoend\\research'

In [2]:
os.chdir("..")
%pwd

'd:\\Code\\mlops\\10.endtoend\\endtoend'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    
    
    alpha: float
    l1_ratio: float
    target_column: str


In [8]:
from box import ConfigBox

from src.datasci.constants import (CONFIG_FILE_PATH, PARAMS_FILE_PATH,
                                   SCHEMA_FILE_PATH)
from src.datasci.utils.common import create_directories, read_yaml


class ConfiguratonManager:
    def __init__(self,
                 config_file_path=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH,
                 schema_file_path=SCHEMA_FILE_PATH
                 ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        create_directories([self.config.artifacts_root])
    
    def get_trainer_config(self)-> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_config= ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name
        )
        return model_config
        
        


In [10]:
import pandas as pd
from sklearn.linear_model import ElasticNet
import joblib

class ModelTraining:
    def __init__(self,config:ModelTrainerConfig):
        self.config=config
    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        
        train_x = train_data.drop([self.config.target_column],axis=1)
        test_x = test_data.drop([self.config.target_column],axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]
        
        model = ElasticNet(alpha=self.config.alpha,l1_ratio=self.config.l1_ratio,random_state=42)
        model.fit(train_x,train_y)
        
        joblib.dump(model,os.path.join(self.config.root_dir,self.config.model_name))
        
        
        

In [11]:
config = ConfiguratonManager()
train_config = config.get_trainer_config()
trainer = ModelTraining(config=train_config)
trainer.train()

[2024-12-07 02:17:22,525: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-07 02:17:22,530: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-07 02:17:22,537: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-07 02:17:22,540: INFO: common: created directory at: artifacts]
[2024-12-07 02:17:22,543: INFO: common: created directory at: artifacts/model_trainer]
